In [1]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re
import os
from bs4 import BeautifulSoup

In [2]:
# Khởi tạo trình duyệt Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") 
options.add_argument("--disable-blink-features=AutomationControlled")  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [3]:
# Đường dẫn để lưu các file SVG
svg_output_dir = "D:\BaiDoAnChuyenNganh3\Automated-Resume-Ranking-System-main\svg_files"
if not os.path.exists(svg_output_dir):
    os.makedirs(svg_output_dir)

In [4]:
# Đọc danh sách link từ file cv_links.csv
input_csv = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv_links.csv"
output_csv = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/final_resumes.csv"

In [5]:
data = []

In [6]:
# Đọc file và crawl dữ liệu
with open(input_csv, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Bỏ qua tiêu đề
    for idx, (category, resume_link) in enumerate(reader, start=1):
        try:
            # Kiểm tra và sửa lỗi link nếu cần
            if not resume_link.startswith(("http://", "https://")):
                resume_link = "https://" + resume_link

            print(f"🔍 Đang truy cập: {resume_link}")
            driver.get(resume_link)

            # Đợi thẻ <svg> xuất hiện
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "svg"))
            )

            resume_text = "N/A"
            resume_html_path = "N/A"

            # 🚀 Xử lý riêng cho file SVG
            if resume_link.endswith(".svg"):
                try:
                    print(f"📄 Đang lấy nội dung file SVG: {resume_link}")
                    
                    # Lấy toàn bộ nội dung của thẻ <svg>
                    svg_element = driver.find_element(By.TAG_NAME, "svg")
                    svg_html = svg_element.get_attribute("outerHTML")  # Lưu toàn bộ mã SVG

                    # Lưu SVG vào file riêng
                    svg_filename = f"resume_{idx}.svg"
                    svg_filepath = os.path.join(svg_output_dir, svg_filename)
                    with open(svg_filepath, "w", encoding="utf-8") as svg_file:
                        svg_file.write(svg_html)
                    resume_html_path = svg_filepath

                    # Phân tích SVG bằng BeautifulSoup để lấy nội dung văn bản
                    soup = BeautifulSoup(svg_html, "xml")
                    
                    # Lấy tất cả các thẻ <text>
                    text_elements = soup.find_all("text")
                    
                    # Sắp xếp các thẻ <text> theo tọa độ y (dòng) và x (vị trí ngang)
                    text_elements_with_coords = []
                    for element in text_elements:
                        transform = element.get("transform", "")
                        match = re.search(r"matrix\([0-1\s]+(-?\d+\.?\d*)\s+(-?\d+\.?\d*)\)", transform)
                        if match:
                            x = float(match.group(1))
                            y = float(match.group(2))
                        else:
                            x, y = 0, 0

                        # Lấy nội dung văn bản từ tất cả <tspan> trong thẻ <text>
                        tspans = element.find_all("tspan")
                        if tspans:
                            text = "".join(tspan.get_text() for tspan in tspans)  # Không strip() ở đây để giữ khoảng trắng
                            last_tspan = tspans[-1]
                            tspan_x_values = last_tspan.get("x", "0").split(",")
                            tspan_x = float(tspan_x_values[-1]) if tspan_x_values else 0
                            # Ước lượng độ dài văn bản (dùng số ký tự nhân với một hệ số)
                            end_x = x + tspan_x + len(last_tspan.get_text()) * 5
                        else:
                            text = element.get_text()
                            end_x = x + len(text) * 5

                        # Bỏ qua các phần tử chứa font-face hoặc base64 (không phải nội dung CV)
                        if text and not re.search(r"@font-face|base64|font-family", text, re.IGNORECASE):
                            text_elements_with_coords.append((y, x, end_x, text))

                    # Sắp xếp theo y (dòng) và x (vị trí ngang)
                    text_elements_with_coords.sort(key=lambda k: (k[0], k[1]))

                    # Gộp văn bản thành các dòng, thêm khoảng cách giữa các từ
                    resume_lines = []
                    current_line = []
                    current_y = None
                    SPACE_THRESHOLD = 10  # Ngưỡng khoảng cách để thêm dấu cách

                    for y, x, end_x, text in text_elements_with_coords:
                        if current_y is None or abs(y - current_y) < 5:  # Cùng dòng (cho phép sai số nhỏ)
                            if current_line:
                                # Tính khoảng cách giữa phần tử hiện tại và phần tử trước đó
                                prev_end_x = current_line[-1][1]  # end_x của phần tử trước
                                gap = x - prev_end_x
                                if gap > SPACE_THRESHOLD:
                                    current_line.append((x, end_x, " "))  # Thêm dấu cách nếu khoảng cách lớn
                            current_line.append((x, end_x, text))
                        else:
                            # Kết thúc dòng hiện tại và bắt đầu dòng mới
                            if current_line:
                                # Gộp các phần tử trong dòng, giữ nguyên khoảng trắng
                                line_text = "".join(item[2] for item in sorted(current_line, key=lambda k: k[0]))
                                # Chuẩn hóa khoảng trắng trong dòng
                                line_text = re.sub(r'\s+', ' ', line_text).strip()
                                if line_text:  # Chỉ thêm dòng nếu không rỗng
                                    resume_lines.append(line_text)
                            current_line = [(x, end_x, text)]
                        current_y = y

                    # Thêm dòng cuối cùng
                    if current_line:
                        line_text = "".join(item[2] for item in sorted(current_line, key=lambda k: k[0]))
                        line_text = re.sub(r'\s+', ' ', line_text).strip()
                        if line_text:
                            resume_lines.append(line_text)

                    # Gộp các dòng văn bản thành một chuỗi
                    resume_text = "\n".join(resume_lines) if resume_lines else "N/A"

                    # In nội dung đã lấy để kiểm tra
                    print(f"📝 Nội dung văn bản (resume_str): {resume_text}")
                    print(f"📝 Đường dẫn SVG (resume_html): {resume_html_path}")

                except Exception as e:
                    print(f"❌ Không thể lấy dữ liệu từ SVG: {e}")
                
            else:
                try:
                    for _ in range(3):
                        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                        time.sleep(2)

                    WebDriverWait(driver, 15).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))
                    )

                    elements = driver.find_elements(By.TAG_NAME, "body") + \
                               driver.find_elements(By.TAG_NAME, "div") + \
                               driver.find_elements(By.TAG_NAME, "p")

                    resume_text = "\n".join([e.text.strip() for e in elements if e.text.strip()])
                    resume_html_path = "N/A"  # Không lưu HTML cho các trang không phải SVG

                except Exception as e:
                    print(f"⚠ Không tìm thấy nội dung trên {resume_link}: {e}")

            data.append([idx, resume_text, resume_html_path, category])
            print(f"✔ Đã lấy dữ liệu: {category}")

        except Exception as e:
            print(f"❌ Lỗi khi crawl {resume_link}: {e}")

🔍 Đang truy cập: https://www.livecareer.com/lcapp/uploads/2024/12/accounting-manager-example-resume.svg
📄 Đang lấy nội dung file SVG: https://www.livecareer.com/lcapp/uploads/2024/12/accounting-manager-example-resume.svg
📝 Nội dung văn bản (resume_str): TAYLOR TRUJILLO
Hialeah, FL 33010• 555-555-5555 • example@example.com
PROFESSIONAL SUMMARY
Dedicated accounting manager with demonstrated history of integrity, efficiency and accuracy in
accounting management. Drive improvements in accounting operations to support organizational
objectives and strategic growth. Proficient in keeping accounts updated and accounting professionals
on-task to handle dynamic conditions. Excellent planning, leadership and decision-making abilities.
WORK HISTORY
Accounting Manager
Altrua Global Solutions - Hialeah, FL 04/2018 - Current
•Spearhead financial operations and lead a team of four accountants to manage day-to-day
accounting activities for a company with $1 million in annual revenue.
•Implement automa

In [7]:
print(data)

[[1, 'TAYLOR TRUJILLO\nHialeah, FL 33010• 555-555-5555 • example@example.com\nPROFESSIONAL SUMMARY\nDedicated accounting manager with demonstrated history of integrity, efficiency and accuracy in\naccounting management. Drive improvements in accounting operations to support organizational\nobjectives and strategic growth. Proficient in keeping accounts updated and accounting professionals\non-task to handle dynamic conditions. Excellent planning, leadership and decision-making abilities.\nWORK HISTORY\nAccounting Manager\nAltrua Global Solutions - Hialeah, FL 04/2018 - Current\n•Spearhead financial operations and lead a team of four accountants to manage day-to-day\naccounting activities for a company with $1 million in annual revenue.\n•Implement automated systems and improve process efficiency, resulting in a 20% reduction in the\nmonthly closing cycle.\n•Achieve a decrease in outstanding accounts receivable by developing and implementing a\nproactive collections strategy.\nSenior Fi

In [8]:
# Đóng trình duyệt
driver.quit()

In [9]:
# Ghi vào file CSV
with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "resume_str", "resume_html", "category"])
    writer.writerows(data)

print(f"✅ Đã lưu kết quả vào {output_csv}")

✅ Đã lưu kết quả vào D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/final_resumes.csv
